In [1]:
pip install tensorflow opencv-python numpy matplotlib


DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/PyAudio-0.2.8-py3.12-macosx-10.9-universal2.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [3]:
import zipfile

# Unzip the dataset
with zipfile.ZipFile('Real_Life_Violence_Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()


In [4]:
!pip install tensorflow tensorflow-gpu numpy matplotlib pillow opencv-python scikit-learn


zsh:1: command not found: pip


In [5]:
!pip3 install tensorflow tensorflow-gpu numpy matplotlib pillow opencv-python scikit-learn


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
Requested tensorflow-gpu from https://files.pythonhosted.org/packages/8a/45/fa31ced1db38f9424f262dfbf35747fe5378b5c808cecb373c8cb8e515d3/tensorflow-gpu-2.12.0.tar.gz has invalid metadata: Expected end or semicolon (after name and no valid version specifier)
    python_version>"3.7"
                  ^
Please use pip<24.1 if you need to use this version.
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu (from versions: 2.12.0)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow-gpu


In [6]:
import os
import cv2
import numpy as np

# Dataset paths
dataset_dir = 'Real_Life_Violence_Dataset/'
violence_dir = os.path.join(dataset_dir, 'Violence')
nonviolence_dir = os.path.join(dataset_dir, 'NonViolence')

# Function to extract frames from a video
def extract_frames(video_path, frames_dir, frame_rate=5):
    video_name = os.path.basename(video_path).split('.')[0]
    frame_dir = os.path.join(frames_dir, video_name)
    os.makedirs(frame_dir, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_rate == 0:  # Capture every nth frame
            cv2.imwrite(os.path.join(frame_dir, f"{frame_count}.jpg"), frame)
        frame_count += 1
    cap.release()

# Extract frames for Violence and NonViolence videos
frames_dir = 'frames'
os.makedirs(frames_dir, exist_ok=True)

violence_frames_dir = os.path.join(frames_dir, 'Violence')
nonviolence_frames_dir = os.path.join(frames_dir, 'NonViolence')
os.makedirs(violence_frames_dir, exist_ok=True)
os.makedirs(nonviolence_frames_dir, exist_ok=True)

for video in os.listdir(violence_dir):
    extract_frames(os.path.join(violence_dir, video), violence_frames_dir)

for video in os.listdir(nonviolence_dir):
    extract_frames(os.path.join(nonviolence_dir, video), nonviolence_frames_dir)


[h264 @ 0x140329490] mb_type 104 in P slice too large at 98 31
[h264 @ 0x140329490] error while decoding MB 98 31


In [8]:
import os
import shutil
import random

# Define directories
frames_dir = 'frames'
validation_dir = 'validation_frames'

# Define subdirectories
subdirs = ['Violence', 'NonViolence']

# Create validation directory structure
for subdir in subdirs:
    os.makedirs(os.path.join(validation_dir, subdir), exist_ok=True)

# Split data into training and validation sets (80% for training, 20% for validation)
for subdir in subdirs:
    frames_path = os.path.join(frames_dir, subdir)
    validation_path = os.path.join(validation_dir, subdir)
    
    # Get list of files
    files = os.listdir(frames_path)
    
    # Split files into training and validation sets
    validation_files = random.sample(files, int(len(files) * 0.2))
    
    # Move validation files to validation directory
    for file in validation_files:
        shutil.move(os.path.join(frames_path, file), validation_path)


In [9]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score, classification_report

# Dataset paths for frames
train_dir = 'frames'
validation_dir = 'validation_frames'  

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary'
)

# Model Definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
    epochs=10
)

# Save Model
model.save('violence_detection_model.h5')


Found 46989 images belonging to 2 classes.
Found 10855 images belonging to 2 classes.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1197s 815ms/step - accuracy: 0.7021 - loss: 0.5525 - val_accuracy: 0.8827 - val_loss: 0.2799
Epoch 2/10
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1s 74us/step - accuracy: 0.9375 - loss: 0.2125 - val_accuracy: 1.0000 - val_loss: 0.2217
Epoch 3/10


2024-11-10 12:46:10.335586: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-11-10 12:46:10.432090: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1090s 742ms/step - accuracy: 0.9138 - loss: 0.2100 - val_accuracy: 0.9098 - val_loss: 0.2644
Epoch 4/10
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1s 42us/step - accuracy: 0.9062 - loss: 0.2451 - val_accuracy: 1.0000 - val_loss: 0.0393
Epoch 5/10


2024-11-10 13:04:21.052211: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1162s 791ms/step - accuracy: 0.9530 - loss: 0.1241 - val_accuracy: 0.9092 - val_loss: 0.3062
Epoch 6/10
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1s 41us/step - accuracy: 0.9688 - loss: 0.0752 - val_accuracy: 0.8571 - val_loss: 0.1896
Epoch 7/10
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1058s 721ms/step - accuracy: 0.9656 - loss: 0.0930 - val_accuracy: 0.9239 - val_loss: 0.2850
Epoch 8/10
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1s 42us/step - accuracy: 0.9688 - loss: 0.0950 - val_accuracy: 0.8571 - val_loss: 0.5730
Epoch 9/10


2024-11-10 13:41:22.711921: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1065s 725ms/step - accuracy: 0.9714 - loss: 0.0799 - val_accuracy: 0.9378 - val_loss: 0.2055
Epoch 10/10
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1s 46us/step - accuracy: 0.9688 - loss: 0.0838 - val_accuracy: 0.8571 - val_loss: 1.1319


In [14]:
!pip install --upgrade pip


zsh:1: command not found: pip


In [15]:
!pip3 install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-24.3.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-24.3.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the Model
model = load_model('violence_detection_model.h5')

# Initialize Webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Pre-process frame (similar to training prep)
    frame_small = cv2.resize(frame, (224, 224))
    frame_small = frame_small / 255.0
    
    # Expand dimensions for model input
    frame_expanded = np.expand_dims(frame_small, axis=0)
    
    # Predict
    predictions = model.predict(frame_expanded)
    prediction = "Violence" if predictions[0][0] > 0.5 else "Non-Violence"
    confidence = predictions[0][0] if predictions[0][0] > 0.5 else 1 - predictions[0][0]
    confidence = round(confidence * 100, 2)
    
    # Display prediction on frame
    cv2.putText(frame, f"{prediction} - {confidence}%", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0) if prediction == "Non-Violence" else (0, 0, 255), 2)
    
    # Display frame
    cv2.imshow('Webcam Feed', frame)
    
    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
